In [1]:
import pandas as pd
import numpy as np

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('corrected_data.csv', skipinitialspace=True, sep=',', index_col='id')
df.head()

,год,время_года,климат,город,страна,способ_охлаждения,режим_при_смешанном_типе_охлаждения,способ_обогрева,возраст,пол,...,скорость_воздуха,рост,вес,занавески,вентилятор,окно,двери,отопление,среднемесячная_температура_на_улице,количество_рекламаций
id,,,,,,,,,,,,,,,,,,,,,
0,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,22.0,NaN,...,0.26,NaN,NaN,1.0,1.0,NaN,NaN,NaN,28.9,0
1,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,40.0,NaN,...,0.12,NaN,NaN,0.0,0.0,NaN,NaN,NaN,84.0,0
2,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,42.0,NaN,...,0.08,NaN,NaN,0.0,0.0,NaN,NaN,NaN,32.8,0
3,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,27.0,NaN,...,27.16,NaN,NaN,1.0,1.0,NaN,NaN,NaN,28.9,1
4,2011,Лето,Cубтропический океанический,Техас,США,Кондиционирование,NaN,NaN,46.0,NaN,...,63.83,NaN,NaN,1.0,1.0,NaN,NaN,NaN,32.8,0


In [4]:
df['режим_при_смешанном_типе_охлаждения'] = df['режим_при_смешанном_типе_охлаждения'].fillna('нет')
df['способ_обогрева'] = df['способ_обогрева'].fillna('нет')

In [5]:
gb = df.groupby(['климат','способ_охлаждения','способ_обогрева'])['оценка_комфорта']
md = gb.median()
for climat, cooling, heating in md.index:
    if not md.isna()[climat, cooling, heating]:
        df['оценка_комфорта'][(df['климат'] == climat) & (df['способ_охлаждения'] == cooling) & (df['способ_обогрева'] == heating)] = md[climat, cooling, heating]
    else:
        df['оценка_комфорта'][(df['климат'] == climat) & (df['способ_охлаждения'] == cooling) & (df['способ_обогрева'] == heating)] = df['оценка_комфорта'].median()

In [10]:
gb = df.groupby('город')['возраст']
md = gb.median()
for city in md.index:
    if not md.isna()[city]:
        df['возраст'][df['город'] == city] = md[city]
    else:
        df['возраст'][df['город'] == city] = df['возраст'].median()

In [49]:
gb = df.groupby('город')['пол']
rel = gb.apply(lambda x: x.value_counts()['Мужской'] / x.value_counts()['Женский'] if x.value_counts().index.size else 0).sum() / gb.apply(lambda x: x.isna().sum() == 0).sum()
for key, it in gb:
    ln = len(it)
    f = int(round(ln / (rel + 1)))
    m = ln - f
    df['пол'].fillna({it.index[i]: ('Мужской' if i < m else 'Женский') for i in range(ln)}, inplace=True)

In [ ]:
for key, it in gb:
    

In [51]:
df.to_csv('corrected_data.csv')